In [1]:
import geopandas as gpd
import pandas as pd

## Open GIS Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Age data

In [3]:
disability_df = pd.read_csv(
    "data/ACSST5Y2023.S1810-2025-05-07T192600.csv", thousands=","
)

In [4]:
disability_df = disability_df.T.reset_index()
disability_df.columns = disability_df.iloc[0].str.rstrip().str.lstrip()
disability_df = disability_df.loc[1:]

In [5]:
disability_percent_df = disability_df[
    disability_df["Label (Grouping)"].str.split("!!").str[1]
    == "Percent with a disability"
]
disability_percent_df["NAME"] = (
    disability_percent_df["Label (Grouping)"].str.split("!!").str[0]
)
disability_percent_df["PERCENT"] = (
    disability_percent_df["With a vision difficulty"].str.replace("%", "").astype(float)
)

/tmp/ipykernel_5407/3013821980.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disability_percent_df["NAME"] = (
/tmp/ipykernel_5407/3013821980.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disability_percent_df["PERCENT"] = (


In [6]:
disability_percent_df = disability_percent_df[["NAME", "PERCENT"]].reset_index(
    drop=True
)

## Combine Data

In [7]:
disability_percent_gdf = states_df.merge(disability_percent_df, on="NAME", how="left")

In [8]:
disability_percent_gdf = disability_percent_gdf[
    ["geometry", "NAME", "PERCENT"]
].dropna()

In [9]:
disability_percent_gdf = disability_percent_gdf.to_crs(9311)

In [10]:
disability_percent_gdf.to_file("data/disabilities.gpkg")